In [ ]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

# Read ground truth shapefile
ground_truth = gpd.read_file('./Shapefiles/ground_truth.shp', dtype={'NZ_time':'str'})

# Read detection shapefile
detections = gpd.read_file('./Shapefiles/detectction_time_filtered.shp', dtype={'DATE':'str'})
# detections

ground_truth_df = ground_truth[['Track_ID', 'time', 'geometry', ]]
ground_truth_df['time'] = pd.to_datetime(ground_truth_df['time'])

detections_df = detections[['True_Track', 'time', 'geometry']]
detections_df['time'] = pd.to_datetime(detections_df['time'])
detections_df['Track_ID'] = detections_df['True_Track']

# Create a new GeoDataFrame to store distances
distances = gpd.GeoDataFrame(columns=['Track_ID', 'time', 'distance', 'geometry'], geometry='geometry')

# Group by Track_ID in ground_truth_df
grouped_ground_truth = ground_truth_df.groupby('Track_ID')

for track_id, group in grouped_ground_truth:
    # Get the corresponding detections for the current Track_ID
    same_track_detections = detections_df[detections_df['Track_ID'] == track_id]
    
    for index, row in group.iterrows():
        time = row['time']
        geometry = row['geometry']
        
        same_time_detections = same_track_detections[same_track_detections['time'] == time]
        
        for det_index, det_row in same_time_detections.iterrows():
            distance = geometry.distance(det_row['geometry'])
            distances = distances.append({'Track_ID': track_id, 'time': time, 'distance': distance, 'geometry': geometry}, ignore_index=True)



In [ ]:
distances.to_csv('./distances_filtered.csv')
